# Projeto 2 - Ciência dos Dados - Estudo sobre Ifood

Nome: Lucas Hix

Nome: Andrea Mindlin Tessler

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [6]:
# Importações

import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
from random import shuffle

___
## Autenticando no  Twitter

* Conta - Lucas Hix: ***@Peng1104Oficial***
* Conta - Andrea Tessler: ***@Andrea25519779***

In [7]:
# Dados de autenticação do twitter:

def mkAPI(filename):
    # leitura do arquivo no formato JSON
    
    with open(filename) as file:
        data = json.load(file)
    
    # Configurando a biblioteca. Não modificar
    
    auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
    auth.set_access_token(data['access_token'], data['access_token_secret'])
    
    # Cria um objeto de API
    return tweepy.API(auth)

# Conta @Peng1104Oficial
api_lucas = mkAPI("lucas-auth.pass")

# Conta @Andrea25519779
api_andrea = mkAPI("andrea-auth.pass")

apis = [api_lucas, api_andrea]

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

In [8]:
# Faz a captura dos Tweets

def getTweets(palavra_chave, quantidade_por_api):
    # Lista da captura
    resultado = []
    
    # Para cada api nas apis geradas
    for api in apis:
        # Contagem de Capturas
        contagem = 0
        
        # Para cada Tweet na captura
        for tweet in tweepy.Cursor(api.search, q=palavra_chave, lang="pt", tweet_mode="extended").items():
            # Numero de pedidos = a qunatidade pedida
            if contagem == quantidade_por_api:
                break
            else:
                # Captura o texto do tweet
                texto = tweet.full_text.lower()
                
                # Verifca se a palavra chave se encontra no texto do tweet
                if palavra_chave.lower() in texto:
                    # Remove duplicatas
                    if texto not in resultado:
                        resultado.append(texto)

                contagem = contagem + 1
    
    # Embaralhando as mensagens para reduzir um possível viés
    shuffle(resultado)
    
    return resultado

# Escolha da marca a ser analisada
marca = "ifood"

# Esolha de quantidade por api (no caso quantidade*2)
quantidade_por_api = 500

dados = getTweets(marca, quantidade_por_api)

# Prepara a nomeação dos execels

numero_da_tentativa = 1

# Lista de todos os arquivos no diretorio
for file in os.listdir("."):
    if marca in file:
        numero_da_tentativa = numero_da_tentativa + 1

# Salva os dados em um arquivo excel
file_name = marca + " " + str(numero_da_tentativa) + ".xlsx"

tabela = pd.DataFrame({"Dados Coletados": pd.Series(dados), "Valor Atribuido": [""]*len(dados)})
tabela.to_excel(file_name, index = False)

print("O Excel", file_name, "foi gerado!")

O Excel ifood 5.xlsx foi gerado!


___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [44]:
# Transforma o excel avaliado em um DataFrame
tabela_processada = pd.read_excel("ifood classificado.xlsx")

In [57]:
def limpeza(texto):
    #Remove Menções
    texto = re.sub("@[^ ]*", "", texto)
    #Remove links
    texto = re.sub("http(|s):\/\/[^ ]*", "", texto)
    #Remove Pontuações
    texto = re.sub("[" + string.punctuation + "]+", "", texto)
    #Remove a marca analisada
    return re.sub("{0}".format(marca), " ", texto)

def getProcesso(valor):
    # Separa os dados dado o valor
    valores = tabela_processada[tabela_processada["Valor Atribuido"] == valor]
    
    # Junta todos os tweets separados em uma string
    texto = limpeza(valores["Dados Coletados"].str.cat())
    
    return pd.Series(texto.split()).value_counts(True)

contra = getProcesso(1)
favoravel = getProcesso(0)

In [59]:
# Nova captura de dados
teste = getTweets(marca, quantidade_por_api)

resultado = []

for tweet in teste:
    # Probabilidade de ser contra
    prob_contra = 0
    
    # Probabilidade de ser favoravel
    prob_favoravel = 0
    
    # Separa o texto
    for palavra in limpeza(tweet).split():
        # Verifica se a palavra se encontra no "Contra"
        if palavra in contra:
            prob_contra = prob_contra + contra[palavra]
        # Verifca se a palavra se encontra no "Favoravel"
        if palavra in favoravel:
            prob_favoravel = prob_favoravel + favoravel[palavra]
    
    # Atribui o valor para o tweet
    if prob_contra > prob_favoravel:
        resultado.append(1) # Contra
    elif prob_contra == prob_favoravel:
        resultado.append(3) # Neutro
    else:
        resultado.append(0) # Favoravel

# Salva o teste em um arquivo excel
file_name_teste = "Teste " + marca + " " + str(numero_da_tentativa) + ".xlsx"

tabela = pd.DataFrame({"Dados Coletados": pd.Series(teste), "Valor Atribuido Automatico": resultado,
                       "Valor Atribuido": [""]*len(teste)})

tabela.to_excel(file_name_teste, index = False)

print("O Excel", file_name_teste, "foi gerado!")

O Excel ifood 5.xlsx foi gerado!


___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**